This notebook contains the code to import and clean the dataset for the Tanzanian water pump survey.


In [1]:
import csv 
import logging
import pandas_profiling
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#initialize run parameters

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
              
train_features_url = "http://s3.amazonaws.com/drivendata/data/7/public/4910797b-ee55-40a7-8668-10efd5c1b960.csv"
train_target_url = "http://s3.amazonaws.com/drivendata/data/7/public/0bf8bc6e-30d0-4c50-956a-603fc693d966.csv"
test_data_url = "https://s3.amazonaws.com/drivendata/data/7/public/702ddfc5-68cd-4d1d-a0de-f5f566f76d91.csv"

In [3]:
# read the data
ds_training_features = pd.read_csv(train_features_url, encoding = 'iso-8859-1')
ds_training_target = pd.read_csv(train_target_url, encoding = 'iso-8859-1')


In [4]:
# confirm there is a label for each of the records in the training_features dataset
len(ds_training_features) == len(ds_training_features[ds_training_features.id == ds_training_target.id])

True

In [5]:
# merge the training features with the target label for EDA and confirm we still have the same #observations
ds_original = pd.merge(ds_training_features, ds_training_target, how='inner', on='id')
len(ds_original) == len(ds_training_target) == len(ds_training_features)

True

In [6]:
# review number of observations and features
ds_original.shape

(59400, 41)

In [7]:
# Make a copy of the original dataset, sort the column names for consistency
# and visually examine a sample of the data
ds_clean = ds_original.reindex(columns=sorted(ds_original.columns))
ds_clean.head()

,amount_tsh,basin,construction_year,date_recorded,district_code,extraction_type,extraction_type_class,extraction_type_group,funder,gps_height,id,installer,latitude,lga,longitude,management,management_group,num_private,payment,payment_type,permit,population,public_meeting,quality_group,quantity,quantity_group,recorded_by,region,region_code,scheme_management,scheme_name,source,source_class,source_type,status_group,subvillage,ward,water_quality,waterpoint_type,waterpoint_type_group,wpt_name
0,6000.0,Lake Nyasa,1999,2011-03-14,5,gravity,gravity,gravity,Roman,1390,69572,Roman,-9.856322,Ludewa,34.938093,vwc,user-group,0,pay annually,annually,False,109,True,good,enough,enough,GeoData Consultants Ltd,Iringa,11,VWC,Roman,spring,groundwater,spring,functional,Mnyusi B,Mundindi,soft,communal standpipe,communal standpipe,none
1,0.0,Lake Victoria,2010,2013-03-06,2,gravity,gravity,gravity,Grumeti,1399,8776,GRUMETI,-2.147466,Serengeti,34.698766,wug,user-group,0,never pay,never pay,True,280,NaN,good,insufficient,insufficient,GeoData Consultants Ltd,Mara,20,Other,NaN,rainwater harvesting,surface,rainwater harvesting,functional,Nyamara,Natta,soft,communal standpipe,communal standpipe,Zahanati
2,25.0,Pangani,2009,2013-02-25,4,gravity,gravity,gravity,Lottery Club,686,34310,World vision,-3.821329,Simanjiro,37.460664,vwc,user-group,0,pay per bucket,per bucket,True,250,True,good,enough,enough,GeoData Consultants Ltd,Manyara,21,VWC,Nyumba ya mungu pipe scheme,dam,surface,dam,functional,Majengo,Ngorika,soft,communal standpipe multiple,communal standpipe,Kwa Mahundi
3,0.0,Ruvuma / Southern Coast,1986,2013-01-28,63,submersible,submersible,submersible,Unicef,263,67743,UNICEF,-11.155298,Nanyumbu,38.486161,vwc,user-group,0,never pay,never pay,True,58,True,good,dry,dry,GeoData Consultants Ltd,Mtwara,90,VWC,NaN,machine dbh,groundwater,borehole,non functional,Mahakamani,Nanyumbu,soft,communal standpipe multiple,communal standpipe,Zahanati Ya Nanyumbu
4,0.0,Lake Victoria,0,2011-07-13,1,gravity,gravity,gravity,Action In A,0,19728,Artisan,-1.825359,Karagwe,31.130847,other,other,0,never pay,never pay,True,0,True,good,seasonal,seasonal,GeoData Consultants Ltd,Kagera,18,NaN,NaN,rainwater harvesting,surface,rainwater harvesting,functional,Kyanyamisa,Nyakasimbi,soft,communal standpipe,communal standpipe,Shuleni


# Drop features with one / mostly one value

In [8]:
# public meeting is True 85% of the time
ds_clean['public_meeting'].value_counts(normalize=True, dropna=False)

True     0.858771
False    0.085101
NaN      0.056128
Name: public_meeting, dtype: float64

In [9]:
ds_clean.drop(columns=['public_meeting'], axis=1, inplace=True)

In [10]:
# quality_group is 'good' 85% of the time
ds_clean['quality_group'].value_counts(normalize=True, dropna=False)

good        0.855522
salty       0.087458
unknown     0.031582
milky       0.013535
colored     0.008249
fluoride    0.003653
Name: quality_group, dtype: float64

In [11]:
ds_clean.drop(columns=['quality_group'], axis=1, inplace=True)

# Drop duplicated features

In [12]:
# quantity & quantity group have identical information, dropping quantity_group
'quantity and quantity_group percent same: {:.2%}'.format((len(ds_clean[ds_clean['quantity'] == ds_clean['quantity_group']])) /len(ds_clean))

'quantity and quantity_group percent same: 100.00%'

In [13]:
ds_clean.drop(columns=['quantity_group'], axis=1, inplace=True)

In [14]:
# payment_type & payment have same with some syntax differences, dropping payment
ds_clean.groupby(['payment_type', 'payment'])['id'].count()

payment_type  payment              
annually      pay annually              3642
monthly       pay monthly               8300
never pay     never pay                25348
on failure    pay when scheme fails     3914
other         other                     1054
per bucket    pay per bucket            8985
unknown       unknown                   8157
Name: id, dtype: int64

In [15]:
ds_clean.drop(columns=['payment'], axis=1, inplace=True)

# Drop features with high rate of missing values

In [16]:
# generate NaN report - many thanks to my Springboard colleague Charles Tucker for this method!

cols = list(ds_clean.columns.values)
nan_dict = {'column_name': list(cols)}

pct_list = []
sum_list = []

# generate a list of the pct null values
for col in cols:
    pct_list.append(ds_clean[col].isnull().mean())
    sum_list.append(ds_clean[col].isnull().sum())
    
# assign lists to the dictionary
nan_dict['pct_missing'] = pct_list
nan_dict['sum_missing'] = sum_list

# convert to dataframe
nan_report = pd.DataFrame(nan_dict)
nan_report.sort_values(by=['pct_missing'], ascending=False, inplace=True)
nan_report[nan_report['sum_missing']>0]

,column_name,pct_missing,sum_missing
26,scheme_name,0.474175,28166
25,scheme_management,0.065269,3877
11,installer,0.061532,3655
8,funder,0.061195,3635
19,permit,0.051448,3056
31,subvillage,0.006246,371


In [17]:
# scheme_name is missing in almost half of the observations, so drop this feature
ds_clean.drop(columns=['scheme_name'], axis=1, inplace=True)

In [18]:
# Permit is boolean - fill in the missing values with true/false values in same proportion as 95% of the data

ds_clean['permit'] = ds_clean['permit'].fillna(pd.Series(np.random.choice([True, False], 
                                                      p=[0.67, 0.33], size=len(ds_clean)))).astype(int)

ds_clean['permit'].value_counts(normalize=True, dropna=False)

1    0.688923
0    0.311077
Name: permit, dtype: float64

We will cover what to do with the rest after reviewing the Pandas Profiling report.

# Drop features with high rate of zero

In [19]:
# generate zeros report for numerical columns to see how many values are zero

cols = ['amount_tsh', 'construction_year', 'gps_height', 'num_private', 'population', 'quantity']
zero_dict = {'column_name': list(cols)}

pct_list = []

# generate a list of the pct null values
for col in cols:
    pct_list.append(len(ds_clean[ds_clean[col]==0]) / len(ds_clean))
    
# assign lists to the dictionary
zero_dict['pct_zero'] = pct_list

# convert to dataframe
zero_report = pd.DataFrame(zero_dict)
zero_report.sort_values(by=['pct_zero'], ascending=False, inplace=True)
zero_report

,column_name,pct_zero
3,num_private,0.987256
0,amount_tsh,0.700993
4,population,0.359949
1,construction_year,0.348636
2,gps_height,0.344074
5,quantity,0.000000


In [20]:
# drop the features with the highest percentages of zero
ds_clean.drop(columns=['amount_tsh', 'num_private'], axis=1, inplace=True)

# of note, population and construction_year are missing in about 1/3 of the observations -
# consider dropping either these features or missing observations in feature selection phase

# leaving gps_height as-is since 0 is a valid value for pumps at sea level!

# Drop features unique to pump & related to surveying

In [21]:
ds_clean.drop(columns=['date_recorded', 'recorded_by', 'wpt_name'], axis=1, inplace=True)

# Examine data types

In [22]:
# the dataset now has 31 columns with no null entries.  There are 7 numerical columns and the remainder are
# categorial. 

ds_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 31 columns):
basin                    59400 non-null object
construction_year        59400 non-null int64
district_code            59400 non-null int64
extraction_type          59400 non-null object
extraction_type_class    59400 non-null object
extraction_type_group    59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
id                       59400 non-null int64
installer                55745 non-null object
latitude                 59400 non-null float64
lga                      59400 non-null object
longitude                59400 non-null float64
management               59400 non-null object
management_group         59400 non-null object
payment_type             59400 non-null object
permit                   59400 non-null int64
population               59400 non-null int64
quantity                 59400 non-null object
re

# Pandas Profiling, feature selection and cardinality reduction

Now that the features with visually-obvious characteristics have been reviewed, run Pandas profiling to get a detailed report on each feature.

Given that most of the features in this dataset are categorical, they will need to be one-hot-encoded for use by the machine learning models, which will create x(n-1) features for x features with n values. This makes reducing cardinality important.  At the same time, there are many features with varying levels of granularity for the same data.  For example, there are 6 features related to the location of the pump. We are looking at the curse of dimensionality.<br><br>
Removing features and reducing cardinality will reduce the variability of the data, but this could lead to an increase in bias.<br><br>
In an attempt to balance the bias/variance tradeoff, two datasets will be created. The first dataset will have a set of features reduced to high-level values with less granularity (for example ‘extraction_type_class’ with 7 unique values). The second dataset will contain features reduced to lower-level values with more granularity (for example ‘extraction_type’ with 18 unique values).


In [23]:
ds_high_var = ds_clean.copy()
ds_low_var = ds_clean.copy()

In [24]:
# Commenting this out once it's run
#run_Pandas_Profiling(ds_clean, break_into_sections = True, num_cols=5)

#### Review results


<b>basin</b> contains 9 unique values - use one-hot encoding to convert to numeric

<b>district_code</b> contains 20 unique values, and is one of 6 features (not including lat,long) indicating pump location. Review with pump location features and decide which to keep.

<b>extraction_type</b> contains 18 unique values, and is one of 3 features indicating pump type. 

<b>extraction_type_class</b> contains 7 unique values, and is one of 3 features indicating pump type. 

<b>extraction_type_group</b> contains 13 unique values, and is one of 3 features indicating pump type.

This extraction feature hierarchy will need to be reduced to one feature when training the algorithms.

In [25]:
ds_clean.groupby(['extraction_type_class','extraction_type_group', 'extraction_type'])['id'].count()

extraction_type_class  extraction_type_group  extraction_type          
gravity                gravity                gravity                      26780
handpump               afridev                afridev                       1770
                       india mark ii          india mark ii                 2400
                       india mark iii         india mark iii                  98
                       nira/tanira            nira/tanira                   8154
                       other handpump         other - mkulima/shinyanga        2
                                              other - play pump               85
                                              other - swn 81                 229
                                              walimi                          48
                       swn 80                 swn 80                        3670
motorpump              mono                   mono                          2865
                       other motorpum

We can see that the largest pump type, gravity, is not broken into any sublevels.  Also, the "handpump" group is the only one that increases in granularity, but most of the values are repeated in the subsequent levels, with only a small number of pumps labeled at the lowest level.  For these reasons, extraction_type_group and extraction_type will be dropped from the low-var dataset, and extraction_type_group will be used in the high-var dataset.

In [26]:
#use extraction_type_class in low_var
ds_low_var.drop(columns=['extraction_type_group', 'extraction_type'], index=1, inplace=True)
#use extraction_type_group in high_var
ds_high_var.drop(columns=['extraction_type_class', 'extraction_type'], index=1, inplace=True)

With the simplified pump type, we can also see an opportunity to reduce the cardinality of the feature by combining the two smallest categories into the 'other' category.

In [27]:
ds_low_var['extraction_type_class'].value_counts(normalize=True)

gravity         0.450833
handpump        0.277042
other           0.108251
submersible     0.104025
motorpump       0.050287
rope pump       0.007593
wind-powered    0.001970
Name: extraction_type_class, dtype: float64

In [28]:
ds_low_var.loc[
    (ds_low_var['extraction_type_class'] == 'rope pump') | 
    (ds_low_var['extraction_type_class'] == 'wind-powered'), 'extraction_type_class'] = 'other'
ds_low_var['extraction_type_class'].value_counts()

gravity        26779
handpump       16456
other           6998
submersible     6179
motorpump       2987
Name: extraction_type_class, dtype: int64

<b>funder</b> contains 1897 unique values, dropping.

In [29]:
ds_low_var.drop(columns=['funder'], axis=1, inplace=True)
ds_high_var.drop(columns=['funder'], axis=1, inplace=True)

<b>gps_height</b>, the altitude of the well, contains 34% zero/missing observations.

<b>installer</b> contains 2146 unique values.

In [30]:
# try generalizing data by grouping on 1st 3 characters
test = pd.DataFrame()
test['new_col'] = ds_clean['installer'].astype(str).str[0:3]
print('Percent of observations covered with top 10 3-character grouping: {:.2%}'.format(test['new_col'].value_counts().head(10).sum() / len(ds_clean)))


Percent of observations covered with top 10 3-character grouping: 54.85%


Half of the observations are covered by generalizing the installer, leaving over 1,000 remaining values. This feature will be dropped.

In [31]:
ds_low_var.drop(columns=['installer'], axis=1, inplace=True)
ds_high_var.drop(columns=['installer'], axis=1, inplace=True)

<b>management</b> has 12 observations, and is a sub-group of <b>management_group</b>, with 5 observations.

In [32]:
ds_clean.groupby(['management_group', 'management'])['id'].count()

management_group  management      
commercial        company               685
                  private operator     1971
                  trust                  78
                  water authority       904
other             other                 844
                  other - school         99
parastatal        parastatal           1768
unknown           unknown               561
user-group        vwc                 40507
                  water board          2933
                  wua                  2535
                  wug                  6515
Name: id, dtype: int64

The vast majority of the data has the same values in the management_group and management features, 'user-group' and 'vwc', so no granularity is gained from the management feature for most of the data.

In [33]:
ds_clean[ds_clean['management_group']=='user-group'].groupby(['status_group'])['id'].count()


status_group
functional                 28252
functional needs repair     3906
non functional             20332
Name: id, dtype: int64

In addition, the values for the target variable are split fairly evenly for the most prevalent group. This might not be enough variance to add value predictions.  We'll keep this in mind during the model tuning phase.  For now, we can drop the "management" feature.

In [34]:
ds_low_var.drop(columns=['management'], axis=1, inplace=True)
ds_high_var.drop(columns=['management'], axis=1, inplace=True)

We can also reduce the cardinality of the "management_group" feature by combining 'unknown' with 'other'

In [35]:
ds_low_var.loc[
    (ds_low_var['management_group'] == 'unknown'), 'management_group'] = 'other'
ds_low_var['management_group'].value_counts()

user-group    52489
commercial     3638
parastatal     1768
other          1504
Name: management_group, dtype: int64

<b>payment_type</b> has 7 values, but 70% of observations are covered in 3 values. 

In [36]:
# To reduce cardinality, update observations that do not have a payment type in the top 3 values 
# to the value of 'other'

ds_low_var.loc[
    (ds_low_var['payment_type'] != 'never pay') & 
    (ds_low_var['payment_type'] != 'per bucket') & 
    (ds_low_var['payment_type'] != 'monthly'), 'payment_type'] = 'other'
ds_low_var['payment_type'].value_counts()

never pay     25347
other         16767
per bucket     8985
monthly        8300
Name: payment_type, dtype: int64

<b>population</b> has 36% missing values

In [37]:
# The population feature also has a high standard deviation.  
ds_clean['population'].describe()

count    59400.000000
mean       179.909983
std        471.482176
min          0.000000
25%          0.000000
50%         25.000000
75%        215.000000
max      30500.000000
Name: population, dtype: float64

In [38]:
# Population can be interpreted as a characteristic of the location of the pump, 
# which is already represented in other features with more complete representation
# in the data. Dropping population.

ds_low_var.drop(columns=['population'], axis=1, inplace=True)
ds_high_var.drop(columns=['population'], axis=1, inplace=True)

<b>quality</b> has 21 different values. 65% of data is represented in top 10 regions. Revisit this when deciding which feature(s) to identify location.

<b>region</b> has 21 different values. 65% of data is represented in top 10 regions. Revisit this when deciding which feature(s) to identify location.

<b>region_code</b> has 27 different values. 65% of data is represented in top 10 region codes. Revisit this when deciding which feature(s) to identify location.

<b>scheme_management</b> has 13 different values. 62% of data is represented in one value, with the rest of the data sparsely distributed over other values. Feature is missing from 6% of observations. Dropping this feature.

In [39]:
ds_low_var.drop(columns=['scheme_management'], axis=1, inplace=True)
ds_high_var.drop(columns=['scheme_management'], axis=1, inplace=True)

<b>source</b> has 10 different values. 95% of data is represented in top 5 sources. 

In [40]:
ds_clean['source'].value_counts()[:5].sum() / len(ds_clean)

0.9566835016835017

In [41]:
# To reduce cardinality, update observations that do not have a sourse in the top 5 values 
# to the value of 'other'

ds_low_var.loc[
    (ds_low_var['source'] != 'spring') & 
    (ds_low_var['source'] != 'shallow well') & 
    (ds_low_var['source'] != 'machine dbh') &
    (ds_low_var['source'] != 'river'), 'source'] = 'other'
ds_low_var['source'].value_counts()

spring          17021
shallow well    16824
machine dbh     11075
river            9612
other            4867
Name: source, dtype: int64

<b>source_class</b> has 3 different values. 77% of values are 'groundwater'. This feature is highly correlated with source, which has more balanced cardinality. Dropping source_class. 

In [42]:
ds_low_var.drop(columns=['source_class'], axis=1, inplace=True)
ds_high_var.drop(columns=['source_class'], axis=1, inplace=True)

<b>source_type</b> has 7 different values, where the majority of data is equal to the values in the <b>source</b> feature. Dropping source_type. 

In [43]:
ds_low_var.drop(columns=['source_type'], axis=1, inplace=True)
#since most of the data is equal to the source feature, that will be used in both datasets
ds_high_var.drop(columns=['source_type'], axis=1, inplace=True)

<b>subvillage</b> has 19288 different values. These are proper names, and are part of the hierarchy of features that identify the location for the pump. A feature with lower cardinality will be better for modeling. Dropping subvillage.

In [44]:
ds_low_var.drop(columns=['subvillage'], axis=1, inplace=True)
#this extreme level of cardinality will most likely not help
ds_high_var.drop(columns=['subvillage'], axis=1, inplace=True)

<b>ward</b> has 2092 different values. These are proper names, and are part of the hierarchy of features that identify the location for the pump. A feature with lower cardinality will be better for modeling. Dropping ward.

In [45]:
ds_low_var.drop(columns=['ward'], axis=1, inplace=True)
#this extreme level of cardinality will most likely not help
ds_high_var.drop(columns=['ward'], axis=1, inplace=True)

<b>water_quality</b> is 'soft' 85% of the time. Dropping water_quality.

In [46]:
ds_low_var.drop(columns=['water_quality'], axis=1, inplace=True)
ds_high_var.drop(columns=['water_quality'], axis=1, inplace=True)

<b>waterpoint_type</b> has 7 values, but 98% of data is in top 4 values (including 'other'). To reduce cardinality, make all but the top 4 values = 'other'. 

In [47]:
ds_clean['waterpoint_type'].value_counts().head(4).sum() / len(ds_clean)

0.9847306397306397

In [48]:
ds_low_var.loc[
    (ds_low_var['waterpoint_type'] != 'communal standpipe') & 
    (ds_low_var['waterpoint_type'] != 'hand pump') & 
    (ds_low_var['waterpoint_type'] != 'communal standpipe multiple') &
    (ds_low_var['waterpoint_type'] != 'other'), 'waterpoint_type'] = 'other'
ds_low_var['waterpoint_type'].value_counts()

communal standpipe             28521
hand pump                      17488
other                           7287
communal standpipe multiple     6103
Name: waterpoint_type, dtype: int64

<b>waterpoint_type_group</b> has 6 values representing the same data in waterpoint_type, where 'communal standpipe' is combined with 'communal standpipe multiple'. Dropping waterpoint_type_group in favor of waterpoint_type, but if dimensionality becomes an issue, use waterpoint_type_group instead of waterpoint_type.

In [49]:
ds_low_var.drop(columns=['waterpoint_type_group'], axis=1, inplace=True)
ds_high_var.drop(columns=['waterpoint_type_group'], axis=1, inplace=True)

## <b>Feature(s) to identify location</b>

In [50]:
df_geo = ds_clean.groupby(['region','region_code','district_code', 'lga'])['id'].count()
df_geo.sort_values


<bound method Series.sort_values of region  region_code  district_code  lga         
Arusha  2            1              Monduli          189
                     2              Arusha Rural    1206
                     3              Arusha Rural      46
                                    Arusha Urban      63
                     5              Ngorongoro       201
                                                    ... 
Tanga   4            6              Handeni          254
                                    Pangani           12
                     7              Kilindi          127
                     8              Mkinga           288
        5            1              Kilindi           34
Name: id, Length: 176, dtype: int64>

Since there is a high degree of cardinality for 'region' which is at the top of the regional hierarchy, the lower-level features will only add more variability, and predictions might benefit from dropping 'region_code','district_code' and 'lga'.  

In [51]:
# for the low-var dataset, drop all but the low-variance pump location fields
ds_low_var.drop(columns=['region_code', 'district_code', 'lga', 'latitude', 'longitude'], axis=1, inplace=True)

# for the high-var dataset, use the higher-variance pump location fields (avoiding extreme cardinality)
# region_code, latitude and longitude
ds_high_var.drop(columns=['region', 'district_code', 'lga'], axis=1, inplace=True)

Now that the Pandas Profiling results have been reviewed, we can come back to the missing / zero-valued data, of which construction year is left.

In [52]:
# determine if construction year is missing according to any pattern
df_age_status = ds_clean[ds_clean['construction_year']==0][['construction_year', 'status_group']]

df_year_zero = pd.crosstab(index = df_age_status['status_group'], 
            columns = df_age_status['construction_year'], normalize=True) 
df_year_zero

construction_year,0
status_group,
functional,0.509682
functional needs repair,0.086388
non functional,0.403931


In [53]:
# compare this to the distribution of status regardless of year
ds_clean['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

The distribution of missing years is close to the distribution of status groups, so there is no pattern in the missing data in relation to the target. Leaving the values as zero for now, and will revisit when optimizing models.

The average of the altitude is close for all three status groups. This makes the feature a candiate for imputing missing values with the mean.  We can optimize the mean by calculating it per geographic area.

In [54]:
sorted(ds_low_var)

['basin',
 'construction_year',
 'extraction_type_class',
 'gps_height',
 'id',
 'management_group',
 'payment_type',
 'permit',
 'quantity',
 'region',
 'source',
 'status_group',
 'waterpoint_type']

In [55]:
sorted(ds_high_var)

['basin',
 'construction_year',
 'extraction_type_group',
 'gps_height',
 'id',
 'latitude',
 'longitude',
 'management_group',
 'payment_type',
 'permit',
 'quantity',
 'region_code',
 'source',
 'status_group',
 'waterpoint_type']

## Write cleaned datasets to file

In [56]:
# finally drop the ID field, which was helpful in the analysis above
ds_low_var.drop(columns=['id'], axis=1, inplace=True)
ds_high_var.drop(columns=['id'], axis=1, inplace=True)

In [57]:
ds_low_var.to_csv('low_var.csv')
ds_high_var.to_csv('high_var.csv')

In [58]:
print('In low variance dataset, {} features were dropped, resulting in dataset with {} features and {} observations. '
      .format(ds_original.shape[1] - ds_low_var.shape[1], ds_low_var.shape[1], ds_low_var.shape[0]))

In low variance dataset, 29 features were dropped, resulting in dataset with 12 features and 59399 observations. 


In [59]:
print('In high variance dataset, {} features were dropped, resulting in dataset with {} features and {} observations. '
      .format(ds_original.shape[1] - ds_high_var.shape[1], ds_high_var.shape[1], ds_high_var.shape[0]))

In high variance dataset, 27 features were dropped, resulting in dataset with 14 features and 59399 observations. 


### These datasets will be used in the subsequent project phases